In [ ]:
import pandas as pd, datetime as dt
from google.cloud import bigquery
from pathlib import Path

events = pd.read_csv("../data_raw/ceo_events.csv", parse_dates=["event_date"])
client = bigquery.Client(project="bigquery-public-data")

rows=[]
for _, ev in events.iterrows():
    s = (ev.event_date-pd.Timedelta(days=7)).strftime("%Y-%m-%d")
    e = (ev.event_date+pd.Timedelta(days=7)).strftime("%Y-%m-%d")
    q=f"""
    SELECT created_utc, title
    FROM `fh-bigquery.reddit_posts.posts`
    WHERE subreddit='stocks'
      AND created_utc BETWEEN '{s}' AND '{e}'
      AND LOWER(title) LIKE '%{ev.ticker.lower()}%'
    """
    df=client.query(q).to_dataframe()
    df["ticker"]=ev.ticker; df["event_date"]=ev.event_date; rows.append(df)

posts=pd.concat(rows)
Path("../data_raw/reddit").mkdir(exist_ok=True)
posts.to_parquet("../data_raw/reddit/rstocks_posts.parquet")
posts.head()
